In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
#Example ADS-B data gathered from flightradar24
flight = pd.read_csv('data/24299085.csv')

print(list(flight))

In [ ]:
#We can use simple accessor methods to find out useful data
flight['Altitude'].min(),flight['Altitude'].max()

In [ ]:
#We can do more complex operations like finding the difference between each successive pair of value
flight['diff'] = flight['Altitude'].diff()

In [ ]:
flight['diff'].min(),flight['diff'].max()

In [ ]:
#We can also create functions to apply to our data
def energy(speed,altitude,mass,gravity=9.81):
    ke=0.5*mass*speed**2
    pe=mass*gravity*altitude
    return ke+pe

In [ ]:
#Using 5000kg as reasonably wrong answer for Jetstream 31
print(energy(flight['Speed'],flight['Altitude'],5000))

In [ ]:
#We can also apply it in a vectorised fashion on our pandas dataframe which is more efficent if your data is large
mass=5000
gravity=9.81
flight['ke'] = flight.apply(lambda r: 0.5*mass*r['Speed']**2,axis=1)
flight['pe'] = flight.apply(lambda r: gravity*mass*r['Altitude'],axis=1)

In [ ]:
flight.head()

In [ ]:
flight['Datetime'] = pd.to_datetime(flight['UTC'],utc=True)

In [ ]:
fig, axes = plt.subplots(3,1,sharex='all')
axes[0].plot(flight['Datetime'],flight['Altitude'])
axes[0].set_ylabel('Altitude')
axes[1].plot(flight['Datetime'],flight['Speed'])
axes[1].set_ylabel('Speed')
axes[2].plot(flight['Datetime'],flight['Direction'])
axes[2].set_ylabel('Direction')
axes[2].set_xlabel('UTC')
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(4,1,sharex='all')
axes[0].plot(flight['Datetime'],flight['Altitude'])
axes[0].set_ylabel('Alt')
axes[1].plot(flight['Datetime'],flight['Speed'])
axes[1].set_ylabel('Speed')
axes[2].plot(flight['Datetime'],flight['ke'])
axes[2].set_ylabel('KE')
axes[3].plot(flight['Datetime'],flight['pe'])
axes[3].set_ylabel('PE')
axes[3].set_xlabel('UTC')
fig.tight_layout()
plt.show()

In [ ]:
coords = [[float(b) for b in a.split(',')] for a in flight['Position']]
#This is a nest list comphrehension to convert the paired coordinates from their 
# comma seperated string format to a pair of floats

In [ ]:
flight['latitude'] = [a for a,b in coords]

In [ ]:
flight['longitude'] = [b for a,b in coords]

In [ ]:
%pip install -q folium

In [ ]:
import folium

m = folium.Map()
folium.PolyLine(zip(flight['latitude'],flight['longitude']), color="red", weight=2.5, opacity=1).add_to(m)
folium.LayerControl().add_to(m)
m.fit_bounds(m.get_bounds())
m

### Case 1

Cabin pressurisation issue

Incident 29th October 2003

File Format

.csv

Line 

0  = metadata (File date, aircraft reg, etc)

1  = more metadata

2  =  Field names

3  =  Field Abbreviations

4  =  Units

5  =  Data Start

Where a variable is recorded at higher than the frame rate it is presented in multiple sequential columns. 
 
The field name is listed for the first column and subsequent columns have a blank field name.

However on lines 2 and 3 the abbreviation and units for the field are listed for each column.

//////  = nan

XXXXX   = nan

******  = nan

------  = nan

In [ ]:
frame_freq = 0.25 # 1 sub frame per second

flight_data1 = pd.read_csv('data/FDM/FDM_data_01.csv',
                          skiprows=5,
                          na_values=['//////',
                                     'XXXXX',
                                     '******',
                                     '------'],
                          names=['Frame-SF',
                                 'Time',
                                 'Status',
                                 'PRESS ALT (ft)',
                                 'COMP AIRSPD DADC 1 (kts)',
                                 'EXCESS CABIN ALT',
                                 'N1 ENG 1 (% RPM)',
                                 'N1 ENG 2 (% RPM)',
                                 "MASTER WARN'G",
                                 'SPEED BRAKE +0s',
                                 'SPEED BRAKE +0.5s',
                                 'AT #1 ENGAGE',
                                 'AT #2 ENGAGE'],
                        )

In [ ]:
def frame_exact_timing(frame_sf_data,time_data,frame_rate):
    frame_name = frame_sf_data.name
    time_name = time_data.name
    dataframe = pd.DataFrame({frame_name:frame_sf_data,time_name:time_data})
    dataframe['changed'] = dataframe[time_name] == dataframe[time_name].shift(1).fillna(dataframe[time_name])
    min_changes = dataframe[frame_name][dataframe['changed']==False].index
    dataframe['seconds'] = 0 
    first_change=min_changes[0]
    for a in range(1,60):
        try:
            dataframe.loc[first_change-a,'seconds'] = 60-a
        except IndexError as ie:
            print(ie)
                
        for b in min_changes:
            try:
                dataframe.loc[b+a,'seconds'] = a
            except IndexError as ie:
                print(ie)
    
    return dataframe['seconds']

In [ ]:
flight_data1['Frame'] = flight_data1.apply(lambda r : r['Frame-SF'][:5],1)
flight_data1['Subframe'] = flight_data1.apply(lambda r : r['Frame-SF'][6:],1)
flight_data1['s past min'] = frame_exact_timing(flight_data1['Frame-SF'],flight_data1['Time'],frame_freq)
# The flight was 29th October 2003
flight_data1['Time'] = flight_data1.apply(lambda r : '29/10/2003 ' + r['Time'] + ':' +'{:02}'.format(int(r['s past min'])),1)
flight_data1['Time (str)'] = flight_data1['Time']
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
# https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
flight_data1['Datetime'] = pd.to_datetime(flight_data1['Time'], format='%d/%m/%Y %H:%M:%S')
# flight_data['Time'] = pd.to_numeric(flight_data['Time'])
del flight_data1['s past min']

In [ ]:
flight_data1['Time'].head()

In [ ]:
flight_data1['Datetime'].head()

## Try plotting the Altitude and Airspeed using Matplotlib

## Now lets look at using SQL

https://docs.python.org/3/library/sqlite3.html

https://docs.sqlalchemy.org/en/14/dialects/sqlite.html

https://www.designcise.com/web/tutorial/what-is-the-order-of-execution-of-an-sql-query

In [ ]:
import sqlite3
import sqlalchemy

from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
flight_data1.to_sql('flight', con=engine)

SQL syntax is not case sensitive but often it is written with commands all caps.  This is a stylistic preference.  Not a requirement

In [ ]:
engine.execute("SELECT * FROM flight").fetchone()

We can find the unique entries in a particular field

In [ ]:
engine.execute("""SELECT distinct "SPEED BRAKE +0s" from flight;""").fetchall()

In [ ]:
engine.execute("""SELECT distinct "AT #1 ENGAGE" from flight;""").fetchall()

### Case 2

Overspeed issue

Incident 5th September 2004

In [ ]:
frame_freq = 0.25 # 1 sub frame per second

flight_data2 = pd.read_csv('data/FDM/FDM_data_02.csv',
                          skiprows=4,
                          na_values=['//////',
                                     'XXXXX',
                                     '******',
                                     '------'],
                          names=['Frame-SF',
                                 'Time',
                                 'Status',
                                 'FDC Event',
                                 'SPD_MACH_A_M',
                                 'ALT',
                                 'FDC HAAL',
                                 'RALT1',
                                 'FDC Rate of Descent',
                                 'CAS',
                                 'GS',
                                 'HEADING',
                                 'ROLL +0s',
                                 'ROLL +0.5s',
                                 'EPR_1',
                                 'EPR_2',
                                 'PITCH +0s',
                                 'PITCH +0.25s',
                                 'PITCH +0.5s',
                                 'PITCH +0.75s',
                                 'LDG_LH',
                                 'ELEV_L',
                                 'ELEV_R',
                                 'RUDDER +0s',
                                 'RUDDER +0.5s',
                                 'VRTG +0s',
                                 'VRTG +0.125s',
                                 'VRTG +0.25s',
                                 'VRTG +0.375s',
                                 'VRTG +0.5s',
                                 'VRTG +0.625s',
                                 'VRTG +0.75s',
                                 'VRTG +0.875s',
                                 'SP2_L',
                                 'SP4_L',
                                 'SP5_L',
                                 'TAT',
                                 'LATG +0s',
                                 'LATG +0.25s',
                                 'LATG +0.5s',
                                 'LATG +0.75',
                                 'LONG +0s',
                                 'LONG +0.25s',
                                 'LONG +0.5s',
                                 'LONG +0.75s',
                                 'FLAPS',
                                 'SLATS',
                                 'GLID1DOT',
                                 'LOC1DOT',
                                 'ATS_ENGAGED',
                                 'AP_1',
                                 'AP_2',
                                 'FDC Phase of Flight ID',
                                 'REV_DEPLOYD1',
                                 'REV_DEPLOYD2',
                                 'GPWS_M_ON',
                                 'GPWS_VA_ON',
                                 'VMO_MMO_OVS']
                        )

In [ ]:
flight_data2.head()

In [ ]:
normal_accelerations = ['VRTG +0s','VRTG +0.125s','VRTG +0.25s','VRTG +0.375s','VRTG +0.5s','VRTG +0.625s','VRTG +0.75s','VRTG +0.875s']
flight_data2[normal_accelerations].head()